In [2]:
import os
os.environ ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk/21.0.2"
os.environ ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec"

In [3]:
import findspark
findspark.init('/opt/homebrew/Cellar/apache-spark/3.5.1/libexec')

In [4]:
spark_url = "local"

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.master(spark_url).appName("DE3").config("spark.ui.port", "4040").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/22 18:45:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/22 18:45:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local appName=DE3>

In [24]:
data_path = './appstore_games.csv'
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [25]:
df.count()

17007

In [26]:
df = df.na.drop()

In [27]:
df.count()

1937

In [32]:
df.count()

1937

In [36]:
df.show(5)

+--------------------+---------+--------------------+--------------------+--------------------+-------------------+-----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+----------------------------+
|                 URL|       ID|                Name|            Subtitle|            Icon URL|Average User Rating|User Rating Count|Price|    In-app Purchases|         Description|           Developer|          Age Rating|           Languages|                Size|       Primary Genre|              Genres|Original Release Date|Current Version Release Date|
+--------------------+---------+--------------------+--------------------+--------------------+-------------------+-----------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------

In [44]:
df = df.withColumn("Average User Rating",df["Average User Rating"].cast("double"))
df = df.withColumn("Price",df["Price"].cast("double"))

df = df.withColumn("User Rating Count",df["User Rating Count"].cast("int"))
df = df.withColumn("Size",df["Size"].cast("int"))

In [46]:
df

DataFrame[URL: string, ID: int, Name: string, Subtitle: string, Icon URL: string, Average User Rating: double, User Rating Count: int, Price: double, In-app Purchases: string, Description: string, Developer: string, Age Rating: string, Languages: string, Size: int, Primary Genre: string, Genres: string, Original Release Date: string, Current Version Release Date: string]

In [62]:
from pyspark.sql.functions import avg

df.agg(avg("Size").alias("Average Size")).collect()[0][0]

213341097.57801765

In [64]:
from pyspark.sql.functions import max

df.agg(max("Price").alias("Max Price")).collect()[0][0]

1639.0

In [72]:
free_game = df.filter(df.Price == 0.0).agg(avg("Average User Rating").alias("Free Average User Rating")).collect()[0][0]
cost_game = df.filter(df.Price != 0.0).agg(avg("Average User Rating").alias("Free Average User Rating")).collect()[0][0]

print("Free game: ", free_game)
print("Cost game: ", cost_game)

print(f"Cost game has average user rating more than free game > 1: ", cost_game > free_game + 1)

Free game:  4.3129855715871255
Cost game:  4.229007633587786
Cost game has average user rating more than free game > 1:  False


In [74]:
free_game = df.filter(df.Price == 0.0).agg(avg("User Rating Count").alias("user_aating_count")).collect()[0][0]
cost_game = df.filter(df.Price != 0.0).agg(avg("User Rating Count").alias("user_aating_count")).collect()[0][0]

print("Free game: ", free_game)
print("Cost game: ", cost_game)

print(f"Free game has user rating count more than cost game > 1000: ", free_game > cost_game + 1000)

Free game:  9496.05049944506
Cost game:  5023.22962962963
Free game has user rating count more than cost game > 1000:  True


24/04/23 01:57:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 960713 ms exceeds timeout 120000 ms
24/04/23 01:57:43 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/23 01:57:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$